In [ ]:
#! unzip /content/drive/MyDrive/ece285_final_project/lihuayue_575cec0b.zip -d /content/drive/MyDrive/ece285_final_project

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/ece285_final_project")

In [ ]:
from save_model import handle_model
from train import train_model
from dataset import GanDataset

In [ ]:
learning_rate = 0.0002
batch_size = 16
num_epochs = 200

In [ ]:
new_or_old_dataset = "old"
if new_or_old_dataset == "new":
    val_dataset = GanDataset("/content/drive/MyDrive/ece285_final_project/ADE20K_2021_17_01", "val")
    train_dataset = GanDataset("/content/drive/MyDrive/ece285_final_project/ADE20K_2021_17_01", "train")
    val_dataset.load_data()
    train_dataset.load_data()
    handle_model("/content/drive/MyDrive/ece285_final_project/val_dataset.pkl", "save", val_dataset)
    handle_model("/content/drive/MyDrive/ece285_final_project/train_dataset.pkl", "save", train_dataset)
else:
    train_dataset = handle_model("/content/drive/MyDrive/ece285_final_project/train_dataset.pkl", "pick")
    val_dataset = handle_model("/content/drive/MyDrive/ece285_final_project/val_dataset.pkl","pick")
    print(len(train_dataset.dataset))
    print(len(train_dataset.labels))

In [ ]:
save_model_or_not = True
load_model_or_not = True
eval_model_dict = "/content/drive/MyDrive/ece285_final_project/eval_result_collection"
load_gen_dict = "/content/drive/MyDrive/ece285_final_project/generator.pkl"
load_dis_dict = "/content/drive/MyDrive/ece285_final_project/discriminator.pkl"
save_gen_dict = "/content/drive/MyDrive/ece285_final_project/generator.pkl"
save_dis_dict = "/content/drive/MyDrive/ece285_final_project/discriminator.pkl"

In [ ]:
train_model(learning_rate,batch_size,num_epochs,train_dataset,val_dataset,save_model_or_not,load_model_or_not,eval_model_dict,load_gen_dict,load_dis_dict,save_gen_dict,save_dis_dict)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from discriminator import Discriminator
from generator import Generator
from tqdm import tqdm
from eval import evaluate
from save_model import handle_model
from PIL import Image
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator(3, 3)
generator.load_state_dict(torch.load(load_gen_dict))
generator.to(device)
for param in generator.parameters():
  param.to(device)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
original_output_list, target_output_list, eval_output_list = evaluate(generator,val_dataloader)

for idx in range(len(target_output_list)):
  print(idx)
  target_output = target_output_list[idx]
  eval_output = eval_output_list[idx]
  original_output = original_output_list[idx]
  eval_output += 1
  eval_output *= 127.5
  original_output += 1
  original_output *= 127.5
  original_image = Image.fromarray(original_output.byte().permute(1, 2, 0).cpu().numpy(), mode='RGB')
  target_image = Image.fromarray(target_output.byte().permute(1, 2, 0).cpu().numpy(), mode='RGB')
  eval_image = Image.fromarray(eval_output.byte().permute(1, 2, 0).cpu().numpy(), mode='RGB')
  target_path = eval_model_dict + ("/%d_target.png")%(idx)
  eval_path = eval_model_dict + ("/%d_eval.png")%(idx)
  original_path = eval_model_dict + ("/%d_original.png")%(idx)
  target_image.save(target_path)
  eval_image.save(eval_path)
  original_image.save(original_path)